In [1]:
# pip install langchain langchain_community

In [2]:
import openai
import os
import langchain
from dotenv import load_dotenv
from IPython.display import Markdown, display
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SimpleSequentialChain
from langchain.chains import SequentialChain
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.llms import OpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferWindowMemory
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

In [3]:
load_dotenv(override=True)

API_KEY=os.getenv("GROQ_API_KEY")
BASE_URL="https://api.groq.com/openai/v1"

MODEL="llama3-70b-8192"

if not API_KEY:
    print("No API_Key found, Please set the API_KEY.")
    exit(1)
elif API_KEY.strip() != API_KEY:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


In [4]:
llm = ChatOpenAI(api_key=API_KEY, base_url=BASE_URL, model=MODEL, temperature=0.6)

C:\Users\rajan\AppData\Local\Temp\ipykernel_21756\3736855045.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(api_key=API_KEY, base_url=BASE_URL, model=MODEL, temperature=0.6)


In [5]:
response = llm([HumanMessage(content="I want to open a restaurant for Italian Food. Suggest a fancy name for this...")])
print(response.content)


C:\Users\rajan\AppData\Local\Temp\ipykernel_21756\1598641334.py:1: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm([HumanMessage(content="I want to open a restaurant for Italian Food. Suggest a fancy name for this...")])


Buon appetito! Here are some fancy name suggestions for your Italian restaurant:

1. **Bella Vita** - "Beautiful Life" in Italian, evoking the warmth and hospitality of Italy.
2. **Fiorella's** - "Little Flower" in Italian, suggesting a delicate and charming dining experience.
3. **Casa Italiano** - "Italian House" in Italian, conveying a sense of home and tradition.
4. **Ristorante Toscano** - "Tuscan Restaurant" in Italian, highlighting the rich culinary heritage of Tuscany.
5. **La Bella Italia** - "Beautiful Italy" in Italian, celebrating the beauty and diversity of Italian cuisine.
6. **Vino e Vita** - "Wine and Life" in Italian, emphasizing the importance of wine in Italian culture.
7. **Caffè Italiano** - "Italian Coffee" in Italian, hinting at the rich coffee culture of Italy.
8. **Il Palio** - named after the famous horse race in Siena, Italy, suggesting a lively and festive atmosphere.
9. **La Dolce Vita** - "The Sweet Life" in Italian, inspired by the classic Italian film, p

In [6]:
name_prompt_template = PromptTemplate(
    input_variables = ['cuisine'],
    template = "I want to open a restaurant for {cuisine} Food. Suggest a fancy name for this..."
)
name_prompt_template.format(cuisine = "italian")

name_chain = LLMChain(llm=llm, prompt=name_prompt_template)
# name_chain.run("Indian")


food_items_prompt_template = PromptTemplate(
    input_variables = ['restaurant_name'],
    template = """Suggest some menu items for a vegetarian restaurant, {restaurant_name}. Reutrn it as comma separated list..."""
)
food_items_prompt_template.format(restaurant_name = "Tandoor Bites")

food_items_chain = LLMChain(llm=llm, prompt=food_items_prompt_template)
# print(food_items_chain.run("Tandoor Bites"))

C:\Users\rajan\AppData\Local\Temp\ipykernel_21756\990543291.py:7: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  name_chain = LLMChain(llm=llm, prompt=name_prompt_template)


In [7]:
chain = SimpleSequentialChain(chains=[name_chain, food_items_chain])
response=chain.run("Mexican")
# print(display(Markdown(response))) 
# print(response)

C:\Users\rajan\AppData\Local\Temp\ipykernel_21756\2649930681.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response=chain.run("Mexican")


In [8]:
name_prompt_template = PromptTemplate(
    input_variables = ['cuisine'],
    template = "I want to open a restaurant for {cuisine} Food. Suggest a fancy name for this..."
)
name_prompt_template.format(cuisine = "italian")

name_chain = LLMChain(llm=llm, prompt=name_prompt_template, output_key='restaurant_name')
# name_chain.run("Indian")


food_items_prompt_template = PromptTemplate(
    input_variables = ['restaurant_name'],
    template = """Suggest some menu items for a vegetarian restaurant, {restaurant_name}. Reutrn it as comma separated list..."""
)
food_items_prompt_template.format(restaurant_name = "Tandoor Bites")

food_items_chain = LLMChain(llm=llm, prompt=food_items_prompt_template, output_key='menu_items')
# print(food_items_chain.run("Tandoor Bites"))

In [9]:
chain = SequentialChain(
    chains = [name_chain, food_items_chain],
    input_variables = ['cuisine'],
    output_variables = ['restaurant_name', 'menu_items']
)

response = chain({"cuisine": "Indian"})
response
# print(display(Markdown(response["cuisine"])))
# print(display(Markdown(response["restaurant_name"])))
# print(display(Markdown(response["menu_items"])))

C:\Users\rajan\AppData\Local\Temp\ipykernel_21756\3797964989.py:7: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = chain({"cuisine": "Indian"})


{'cuisine': 'Indian',
 'restaurant_name': 'What a great idea! Indian cuisine is loved by many around the world. Here are some fancy name suggestions for your Indian restaurant:\n\n1. **Tandoor Mahal**: "Tandoor" is a traditional Indian clay oven, and "Mahal" means palace, evoking a sense of luxury and grandeur.\n2. **Spice Route**: This name pays homage to the ancient trade routes that brought exotic spices from India to the world, hinting at the rich flavors your restaurant will offer.\n3. **Dhaba Royale**: "Dhaba" is a traditional Punjabi eatery, and adding "Royale" gives it a regal twist, suggesting a refined and upscale dining experience.\n4. **Maharaja\'s Kitchen**: "Maharaja" means great king, implying a majestic and opulent atmosphere, while "Kitchen" emphasizes the heart of the restaurant.\n5. **Garam Glamour**: "Garam" means hot or warm in Hindi, and "Glamour" adds a touch of sophistication, suggesting a vibrant and stylish dining experience.\n6. **Saffron Dreams**: Saffron is

In [10]:
# pip install wikipedia

In [11]:
load_dotenv(override=True)
SERPAPI_API_KEY = os.getenv("SERPAPI_API_KEY")

if not SERPAPI_API_KEY:
    print("No SERPAPI_API_KEY found, Please set the API_KEY.")
    exit(1)
elif SERPAPI_API_KEY.strip() != SERPAPI_API_KEY:
    print("A SERPAPI_API_KEY was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("SERPAPI_API_KEY found and looks good so far!")

SERPAPI_API_KEY found and looks good so far!


In [12]:
# pip install google-search-results

In [13]:
# tools = load_tools(["wikipedia"], llm=llm)
tools = load_tools(["serpapi"], llm=llm)

agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)
# agent.run("What is the projected GDP of India in 2030?")

C:\Users\rajan\AppData\Local\Temp\ipykernel_21756\892433791.py:4: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)


In [14]:
from langchain.memory import ConversationBufferMemory

memory  = ConversationBufferMemory()

chain = LLMChain(llm=llm, prompt=name_prompt_template, memory=memory)
name = chain.run("Indian")
# print(name)

C:\Users\rajan\AppData\Local\Temp\ipykernel_21756\836626835.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory  = ConversationBufferMemory()


In [15]:
name =chain.run('Italian')
# print(name)

In [16]:
# print(chain.memory.buffer)

In [17]:
convo = ConversationChain(llm =llm)
print(convo.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


C:\Users\rajan\AppData\Local\Temp\ipykernel_21756\1010055282.py:1: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :class:`~langchain_core.runnables.history.RunnableWithMessageHistory` instead.
  convo = ConversationChain(llm =llm)


In [18]:
convo.run("Who won the world cup in 2022?")

"The 2022 FIFA World Cup was a thrilling tournament, held in Qatar from November 20 to December 18, 2022. And the winner of that tournament was Argentina! They defeated France 4-2 in a penalty shootout, after the match had ended 3-3 after extra time. It was Argentina's third World Cup title, and their first since 1986. Lionel Messi, arguably one of the greatest footballers of all time, finally got his hands on the coveted trophy in what was likely his last World Cup appearance."

In [19]:
convo.run("Which will be trending technology in 2030...?")

"What a great question! Based on my training data and analysis of current trends, I'd say that some of the trending technologies in 2030 will likely include Extended Reality (XR), which encompasses Virtual Reality (VR), Augmented Reality (AR), and Mixed Reality (MR). We can expect to see significant advancements in XR, with more affordable and accessible devices, leading to increased adoption in various industries such as education, healthcare, and entertainment.\n\nAnother area that's likely to gain traction is Artificial General Intelligence (AGI). While we're still far from achieving true human-like intelligence in machines, researchers are making rapid progress in developing more sophisticated AI models that can perform complex tasks, learn from experience, and adapt to new situations.\n\nQuantum Computing is another technology that's expected to make significant strides in 2030. With companies like Google, IBM, and Microsoft already investing heavily in quantum computing research,

In [20]:
print(convo.memory.buffer)

Human: Who won the world cup in 2022?
AI: The 2022 FIFA World Cup was a thrilling tournament, held in Qatar from November 20 to December 18, 2022. And the winner of that tournament was Argentina! They defeated France 4-2 in a penalty shootout, after the match had ended 3-3 after extra time. It was Argentina's third World Cup title, and their first since 1986. Lionel Messi, arguably one of the greatest footballers of all time, finally got his hands on the coveted trophy in what was likely his last World Cup appearance.
Human: Which will be trending technology in 2030...?
AI: What a great question! Based on my training data and analysis of current trends, I'd say that some of the trending technologies in 2030 will likely include Extended Reality (XR), which encompasses Virtual Reality (VR), Augmented Reality (AR), and Mixed Reality (MR). We can expect to see significant advancements in XR, with more affordable and accessible devices, leading to increased adoption in various industries su

In [21]:
memory = ConversationBufferWindowMemory(k=1)

convo_new = ConversationChain(llm=llm, memory=memory)

C:\Users\rajan\AppData\Local\Temp\ipykernel_21756\2384387795.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(k=1)


In [22]:
convo.run("Who won the world cup in 2022?")

'Déjà vu! I already told you earlier that Argentina won the 2022 FIFA World Cup, defeating France 4-2 in a penalty shootout, after the match had ended 3-3 after extra time!'

In [23]:
convo.run("Which will be trending technology in 2030...?")

"Déjà vu again! I already shared my predictions with you earlier! To recap, I think Extended Reality (XR), Artificial General Intelligence (AGI), Quantum Computing, and the Internet of Bodies (IoB) will be some of the trending technologies in 2030. But if you'd like, I can provide more details or insights on any of these areas!"

In [24]:
convo.run("Who was the captian of the winning team?")

'The captain of the winning team, Argentina, in the 2022 FIFA World Cup was Lionel Messi! He led his team to victory, and it was a fitting end to his World Cup career.'

# **RAG**

In [25]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader(web_paths=["https://www.microwebtec.com/"])
documents = loader.load()
print(documents)

USER_AGENT environment variable not set, consider setting it to identify your requests.


[Document(metadata={'source': 'https://www.microwebtec.com/', 'title': 'Full stack Development Company - Microweb Software Pvt Ltd', 'description': 'Microweb Software, A Full stack Development Company offers Digital transformation services, Web & Mobile Development services & Enterprise Solutions', 'language': 'en'}, page_content='\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nFull stack Development Company - Microweb Software Pvt Ltd\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nHome\nAbout Microweb\nCases\nServices\nTechnology\n\nCloud Application Development Services\nAzure DevOps Services\nGolang Development Services\nDevOps Consulting Services\nBusiness Transformation\nLaravel Application Development\nSymfony Web Development\nNode.js Development\nAngularJs Web Development Services\nRuby on Rails Application Development\nMicrosoft Development\nMobile App

In [26]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(documents)

In [27]:
print(splits[0])
print(splits[1])

page_content='Full stack Development Company - Microweb Software Pvt Ltd
























































































Home
About Microweb
Cases
Services
Technology

Cloud Application Development Services
Azure DevOps Services
Golang Development Services
DevOps Consulting Services
Business Transformation
Laravel Application Development
Symfony Web Development
Node.js Development
AngularJs Web Development Services
Ruby on Rails Application Development
Microsoft Development
Mobile Application Development
IoT and Embedded Systems & Smart Solutions
Cloud Technology
ReactJS Development
Drupal Web Development Services
2D and 3D Video Animation
UI & UX Design
Web Development
Enterprise Solutions
Digital Marketing
Software Outsource to India
Graphic Designing
eCommerce Development
WordPress Web Development
WooCommerce Development
Shopify Development
Python Development
AI & ML Services


Blogs
Contact' metadata={'source': 'https://www.microwebtec.com/',

In [28]:
print(len(splits))

5


In [29]:
# pip install sentence-transformers chromadb tiktoken

In [30]:
from langchain.embeddings import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

vectorstore = Chroma.from_documents(documents=splits, embedding=embedding_model)

C:\Users\rajan\AppData\Local\Temp\ipykernel_21756\2774110483.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


In [31]:
print(vectorstore._collection.count())

5


In [32]:
print(vectorstore._collection.get())

{'ids': ['28786983-aec1-4c52-b982-2ffc1fe937b3', 'c89b29c3-6821-481c-9360-5ad33a3e760e', '37e754c7-2d00-4ecd-aa39-2d2c3918465f', '0172a806-d486-44fc-894c-a80baedcdae9', '41b102c1-8a7f-40c4-a83d-d36033c94c8a'], 'embeddings': None, 'documents': ['Full stack Development Company - Microweb Software Pvt Ltd\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nHome\nAbout Microweb\nCases\nServices\nTechnology\n\nCloud Application Development Services\nAzure DevOps Services\nGolang Development Services\nDevOps Consulting Services\nBusiness Transformation\nLaravel Application Development\nSymfony Web Development\nNode.js Development\nAngularJs Web Development Services\nRuby on Rails Application Development\nMicrosoft Development\nMobile Application Development\nIoT and Embedded Systems & Smart Solutions\nCloud Technology\nReactJS Development\nDrupal Web Development Servic

In [33]:
print("\n Collection 1- ", vectorstore._collection.get(ids=['94c2431d-ff50-431c-8e66-9a6104c320b4'], include=["embeddings", "documents"]))
# print("\n Collection 2- ", vectorstore._collection.get(ids=['2ccbdf71-97e0-487b-bc18-2970c224f49c'], include=["embeddings", "documents"]))
# print("\n Collection 3- ", vectorstore._collection.get(ids=['39e457a8-4912-4e02-8be1-4e7af68cdcab'], include=["embeddings", "documents"]))


 Collection 1-  {'ids': [], 'embeddings': array([], dtype=float64), 'documents': [], 'uris': None, 'included': ['embeddings', 'documents'], 'data': None, 'metadatas': None}


In [34]:
retriever = vectorstore.as_retriever()

In [35]:
from langchain import hub
prompt  = hub.pull("rlm/rag-prompt")

C:\Users\rajan\AppData\Roaming\Python\Python312\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [36]:
# pip install langchain_openai

In [37]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(api_key=API_KEY, base_url=BASE_URL, model=MODEL, temperature=0.6)

In [38]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

rag_chain = ({"context": retriever, "question": RunnablePassthrough})

def format_docs(docs):
    return "\n\n".join([d.page_content for doc in docs])

rag_chain = ({"context": retriever | format_docs, "question": RunnablePassthrough()}
            | prompt
            | llm
            | StrOutputParser())

In [39]:
rag_chain.invoke("Which type of services are there?")

NameError: name 'd' is not defined